In [1]:
import numpy as np
import time
from datetime import timedelta
import math

import datetime
import time

import tensorflow as tf

import pandas as pd
import numpy as np
import keras as keras
import keras.backend as K
from keras.models import Model
from keras.models import load_model

import matplotlib.pyplot as plt

import random
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Using TensorFlow backend.


In [2]:
DATA_PATH = 'data/'

In [3]:
df = pd.read_pickle(DATA_PATH + "data.pkl")
df.Weight = df.Weight.astype(np.float)
df.BlobImages = df.BlobImages.apply(lambda x : x.reshape(27, 15, 1))

In [4]:
train = df.BlobImages.values
train = np.concatenate(train).reshape(-1, 27, 15, 1) / 255.0
print("Done!")

Done!


In [10]:
loadpath = "./ModelSnapshots/model-500.h5"

In [6]:
def rmse(y_true, y_pred):
    return  K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [7]:
# regulate gpu usage
config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)#, device_count = {'GPU' : 4})
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction=0.1
config.gpu_options.allocator_type = 'BFC'
tf.Session(config=config)

In [11]:
model = load_model(loadpath, custom_objects={'rmse': rmse})

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [13]:
%%time
lst = []
for i in range(0, len(train), 10000):
    _x = train[i: i+10000]
    lst.extend(model.predict(_x))

CPU times: user 2min 18s, sys: 13.5 s, total: 2min 31s
Wall time: 1min 26s


In [14]:
df["WeightPred"] = lst
df.WeightPred = df.WeightPred.apply(lambda x: x[0])
df["Error"] = df.Weight - df.WeightPred
df["ErrorAbs"] = df.Error.abs()

In [15]:
mean = df.groupby("Weight").WeightPred.mean()
std = df.groupby("Weight").WeightPred.std()

In [18]:
dfX = df[(df.Set == "Train")]
rmse = np.sqrt(mean_squared_error(dfX.Weight, dfX.WeightPred))
print("# $\\textit{RMSE} = %.1fg$" % rmse)
print("# $\\textit{MAE} = %.1fg$ ($\\textit{SD} = %.1fg$)" % (dfX.ErrorAbs.mean(), dfX.ErrorAbs.std()))

# $\textit{RMSE} = 484.5g$
# $\textit{MAE} = 396.5g$ ($\textit{SD} = 278.3g$)


In [19]:
dfX = df[(df.Set == "Test") & (df.Noise == 0) & (df.Version == "Normal")]
rmse = np.sqrt(mean_squared_error(dfX.Weight.values, dfX.WeightPred.values))
print("# $\\textit{RMSE} = %.2fg$" % rmse)
print("# $\\textit{MAE} = %.2fg$ ($\\textit{SD} = %.2fg$)" % (dfX.ErrorAbs.mean(), dfX.ErrorAbs.std()))


# $\textit{RMSE} = 471.99g$
# $\textit{MAE} = 369.01g$ ($\textit{SD} = 294.30g$)
